# 🧹 Data Cleaning Challenge: Enfrentando o Caos dos Dados Reais

## 1. O Problema
Muitos datasets acadêmicos são entregues "limpos". Na vida real, os dados chegam incompletos, com formatos inconsistentes, duplicados e erros humanos. 

Este projeto demonstra minhas habilidades em **Data Wrangling** e **Data Cleaning**, transformando um CSV caótico em uma base de dados pronta para análise.

## 2. Diagnóstico da Sujeira
Ao carregar o dataset `vendas_brutas_caos.csv`, identificamos os seguintes problemas:
- **Inconsistência de Texto:** Nomes em maiúsculas, minúsculas, com espaços extras e caracteres especiais.
- **Formatos de Data:** Mistura de padrões ISO, brasileiro e erros de digitação (mês 13).
- **Valores Numéricos:** Moeda (R$), separadores de milhar (.), decimais (,) e valores negativos sem sentido.
- **Duplicações:** IDs de transação repetidos com informações conflitantes.
- **Missing Values:** Lacunas críticas em colunas de valor e data.

In [ ]:
import pandas as pd
import numpy as np
import re

# Carregando o caos
df = pd.read_csv('vendas_brutas_caos.csv', encoding='latin-1')
print("Amostra dos dados sujos:")
display(df.head(10))

## 3. Plano de Higienização (Data Wrangling)

### 3.1 Tratamento de Nomes e Strings

In [ ]:
def clean_names(name):
    if pd.isna(name): return "Desconhecido"
    name = str(name).strip().upper() # Padronizar para maiúsculo sem espaços nas bordas
    name = re.sub(' +', ' ', name)   # Remover espaços múltiplos internos
    name = name.replace('_', ' ')     # Corrigir separadores
    return name

df['cliente_nome'] = df['cliente_nome'].apply(clean_names)
df['status'] = df['status'].str.strip().str.upper()

### 3.2 Padronização de Valores Monetários

In [ ]:
def clean_currency(value):
    if pd.isna(value) or value == 'NULL': return 0.0
    v = str(value).replace('R$', '').replace(' ', '')
    
    # Lógica para tratar separador de milhar vs decimal
    if ',' in v and '.' in v:
        v = v.replace('.', '').replace(',', '.')
    elif ',' in v:
        v = v.replace(',', '.')
        
    v_num = pd.to_numeric(v, errors='coerce')
    return abs(v_num) if v_num else 0.0 # Trade-off: assumimos valor absoluto para erros de sinal

df['valor_venda'] = df['valor_venda'].apply(clean_currency)

### 3.3 Conserto de Datas e Duplicados

In [ ]:
# Converter para datetime forçando NaT para erros (como mês 13)
df['data_venda'] = pd.to_datetime(df['data_venda'], errors='coerce', dayfirst=True)

# Preencher datas nulas com a data anterior (forward fill) como política de negócio
df['data_venda'] = df['data_venda'].ffill()

# Remover duplicados de ID mantendo a última ocorrência (supostamente a mais atualizada)
df = df.drop_duplicates(subset='transacao_id', keep='last')

print("Dados Higienizados:")
display(df.head())

## 4. Conclusão e Trade-offs

Neste projeto, tomei as seguintes decisões técnicas:
1. **Nomes:** Optei por converter para Maiúsculas para facilitar joins futuros, mesmo perdendo a capitalização original.
2. **Valores Negativos:** Usei a função `abs()` assumindo que foram erros de entrada de dados (sinal invertido).
3. **Datas Inválidas:** Usei Forward Fill (`ffill`). Em um cenário real, isso dependeria de validar com o time de banco de dados, mas aqui serviu para manter a continuidade da série temporal.

**Resultado:** O dataset agora está 100% tipado e consistente para análises estatísticas.